In [8]:
import pandas as pd

In [9]:
texts = pd.read_csv("./data/posts.tsv", sep="\t", encoding="utf-8").text

In [10]:
from bs4 import BeautifulSoup
import nltk
from nltk.corpus import stopwords
import re

In [11]:
stops = set(stopwords.words("english") + stopwords.words("russian"))

In [12]:
def post_to_words(text):
    lower_words = re.sub(ur"[^а-я]", " ", BeautifulSoup(text).get_text().lower()).split()
    meaningful_words = [w for w in lower_words if not w in stops]
    stemmer = nltk.stem.snowball.RussianStemmer()
    return [stemmer.stem(t) for t in meaningful_words]

In [13]:
processed_texts = map(lambda x: post_to_words(unicode(x)), texts)

/Users/oruen/apps/my/bigdata/lab4/venv/lib/python2.7/site-packages/bs4/__init__.py:189: UserWarning: "https://www.facebook.com/events/406603402849183/411253782384145/?notif_t=plan_mall_activity" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client to get the document behind the URL, and feed that document to Beautiful Soup.
  '"%s" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client to get the document behind the URL, and feed that document to Beautiful Soup.' % markup)


In [14]:
from gensim import corpora, models, similarities

dictionary = corpora.Dictionary(processed_texts)
print 'Original: {}'.format(dictionary)
dictionary.filter_extremes(no_below = 5, no_above = 0.5, keep_n=None)
dictionary.save('nips.dict')
print 'Filtered: {}'.format(dictionary)

Original: Dictionary(11763 unique tokens: [u'\u0432\u044b\u043f\u043e\u043b\u043d', u'\u0443\u043f\u043e\u043c\u044f\u043d\u0443\u0442', u'\u0432\u0437\u043c\u0430\u0445', u'\u043e\u0433\u0440\u0430\u0436\u0434\u0435\u043d', u'\u0444\u0441\u0431\u0443\u0447\u0430\u0441\u0442\u043e\u043a']...)
Filtered: Dictionary(3160 unique tokens: [u'\u0432\u044b\u043f\u043e\u043b\u043d', u'\u043e\u0437\u043d\u0430\u0447\u0430', u'\u0441\u043e\u043c\u043d\u0438\u0442\u0435\u043b\u044c\u043d', u'\u0441\u043c\u0435\u0448\u043d', u'\u043d\u0430\u0446\u0438\u043e\u043d\u0430\u043b\u0438\u0441\u0442']...)


In [15]:
corpus = [dictionary.doc2bow(text) for text in processed_texts]
corpora.MmCorpus.serialize('nips.mm', corpus) # store on disc

In [16]:
print corpus[7][0:20]

[(160, 2), (720, 1), (731, 1), (1040, 2), (1431, 1), (1461, 1), (1480, 1), (1560, 1), (1780, 1), (1792, 1), (1862, 1), (1872, 1), (1887, 1), (1918, 1), (2082, 1), (2422, 1), (2500, 1), (2536, 1), (2538, 1), (2540, 1)]


In [17]:
from time import time
start = time()
model = models.ldamodel.LdaModel(corpus, id2word=dictionary, num_topics=5, chunksize=2000, update_every=1, passes=40, decay=0.5)
print 'Evaluation time: {}'.format((time()-start) / 60)

Evaluation time: 2.31935614745


In [21]:
for position in range(10):
    for topic in range(5):
        print model.show_topic(topic)[position][1].center(15, ' '),
    print

      наш             куб           навальн         навальн            ул      
       эт            газет           команд            эт            выход     
      нам            средн            штаб           котор            метр     
      нужн           разда           приход           сво              д       
      сво           человек           нам             год           проспект   
      очен          волонтер          друз           москв           площад    
      друз            выда            наш             дел             улиц     
     команд         листовок         встреч           росс          бульвар    
      всем          навальн          сегодн         коррупц            м       
    навальн          сегодн          сдела            мэр             куб      


Куб — агитационная точка сторонников Навального.

1. Слегка разплывчатое описание того, чем так хорошо работать в команде Навального.
2. Описание конкретных действий, которыми волонтеры могут помочь команде Навального: раздача газет и листовок, агитация на кубах.
3. Мотивация сторонников Навального помогать агитационному процессу, участие во встречах с Навальным.
4. Тема о том, как Навальный успешно победил бы коррупцию, если бы стал мэром Москвы.
5. В этой теме можно заметить много слов, относящихся к местоположению: улица, проспект, метро, площадь, а также куб. Эта тема — о том, где конкретно проходит агитация и куда можно подойти, чтобы ей помочь. 